In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gp
from os.path import join, basename
from datetime import date, datetime

In [ ]:
import matplotlib.pyplot as plt
from plot_tools import *
import cartopy.crs as ccrs
import seaborn as sns

In [ ]:
root = r'/scratch/compound_hotspots'
ddir = join(root, 'data', '4-postprocessed')
fdir = join(root, 'reports', 'figures')

In [ ]:
rc = {'savefig.bbox': 'tight',  'savefig.format': 'png', 'savefig.dpi':300}
context = 'paper'
# sns.set(context=context, style='whitegrid', font_scale=0.75 if context == 'talk' else 1., rc=rc)
sns.set(context=context, style='whitegrid', font_scale=1.2 if context == 'paper' else 1., rc=rc)

crs = ccrs.Robinson()
crs_sub = ccrs.PlateCarree()

cmap_div = sns.diverging_palette(220, 10, s=75, l=40, sep=1, as_cmap=True)
bmap_kwargs = dict()
#     features=['land', 'rivers'],
#     feat_colors = [cfeature.COLORS['land_alt1'], cfeature.COLORS['water']],
# )

plot_kwargs=dict(edgecolor=(0.5, 0.5, 0.5, 0.8), linewidth=0.5, legend=False, zorder=2)
box_kwargs=dict(whis=[5,95], showfliers=False, boxprops=dict(linewidth=1.5), medianprops=dict(linewidth=1.5))

regions = {
    'A': (-100.0, 22.0, -50.0, 53.0), # 30 x 18
    'B': (-12.0, 35.0, 38.0, 65.0), # 50 x 30
    'C': (100.0, 20.0, 150.0, 50.0),  # 50 x 30
}

In [ ]:
attrs_fn = join(ddir, 'rivmth_mean_attrs.csv')
attrs = pd.read_csv(attrs_fn, index_col='index').rename(columns={'rivmth_lat':'lat', 'rivmth_lon':'lon'})
attrs['Q_mean_log10'] = np.log10(attrs['Q_mean'])
attrs['markersize'] = np.maximum((attrs['Q_mean_log10'] / attrs['Q_mean_log10'].max()),0) * 30 + 15


In [ ]:
fn_rivmth_ts = join(ddir, 'rivmth_reanalysis.zarr')
ds_drivers = xr.open_zarr(fn_rivmth_ts)
for scen in ds_drivers['scen'].data:
    ds_drivers[f'WSE{scen}'] = ds_drivers['WSE'].sel(scen=scen).drop('scen')
    ds_drivers[f'WSE{scen}_amax'] = ds_drivers[f'WSE{scen}'].resample(time='A').max('time').mean('time')
    attrs[f'WSE{scen}_amax'] = ds_drivers[f'WSE{scen}_amax'].mean('ensemble')
ds_drivers['Qmsl_amax'] = ds_drivers['Qmsl'].resample(time='A').max('time').mean('time')
ds_drivers['Qmsl_mean'] = ds_drivers[f'Qmsl'].mean('time')
attrs['Htiderange_day'] = (ds_drivers['Htide_day_max'] - ds_drivers['Htide_day_min']).mean('time').values
attrs['Htiderange_yr'] = (ds_drivers['Htide_day_max'].resample(time='A').max('time') - ds_drivers['Htide_day_min'].resample(time='A').min('time')).mean('time').values
attrs['Hseasrange_yr'] = (ds_drivers['Hseas'].resample(time='A').max('time')-ds_drivers['Hseas'].resample(time='A').min('time')).mean('time').values

## Attrs

In [ ]:
plots = [
    dict(
        column = 'Qmsl_mean',
        unit ='[m3/s]',
        vlim = [-1, 4],
        nticks = 6,
        cmap=plt.cm.viridis,
        fapply=np.log10,
    ),
    dict(
        column = 'Qmsl_amax',
        unit ='[m3/s]',
        vlim = [-1, 4],
        nticks = 6,
        cmap=plt.cm.viridis,
        fapply=np.log10,
    ),
    dict(
        column = 'uparea',
        unit ='[km2]',
        vlim = [3, 6],
        nticks = 4,
        cmap=plt.cm.viridis,
        fapply=np.log10,
    ),
    dict(
        column = 'WSEsurge_amax',
        unit ='[m+EGM96]',
        vlim = [-0.25, 2.75],
        nticks = 13,
        cmap=plt.cm.viridis,
        discrete=False,
    ),
    dict(
        column = 'WSEseas_amax',
        unit ='[m+EGM96]',
        vlim = [-0.25, 2.75],
        nticks = 13,
        cmap=plt.cm.viridis,
        discrete=False,
    ),
    
    dict(
        column = 'WSEtide_amax',
        unit ='[m+EGM96]',
        vlim = [-0.25, 2.75],
        nticks = 13,
        cmap=plt.cm.viridis,
        discrete=False,
    ),
    dict(
        column = 'Hskewsurge_amax',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 1],
        nticks = 21,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'Hsurge_amax',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 1],
        nticks = 21,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'Hseas_amax',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 0.2],
        nticks = 5,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'Hseasrange_yr',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 0.5],
        nticks = 11,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'Htiderange_yr',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 4],
        nticks = 11,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'Htiderange_day',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [0, 4],
        nticks = 11,
        cmap=plt.cm.viridis,
        markersize=20,
    ),
    dict(
        column = 'wse_std',
        x_col='lon', y_col='lat',
        unit ='[m]',
        vlim = [0, 1],
        nticks = 11,
        cmap=plt.cm.viridis,
        markersize=20,
    ),    
    dict(
        column = 'Hseas_amin',
        x_col='gtsm_lon', y_col='gtsm_lat',
        unit ='[m]',
        vlim = [-0.2, 0],
        nticks = 5,
        cmap=plt.cm.viridis_r,
        markersize=20,
    ),
]

In [ ]:

for pdict in plots[-1:]:
#     plt.close('all')
    fig = plt.figure(figsize=(17, 10))
    grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

    column = pdict.get('column')
    x_col=pdict.get('x_col', 'lon')
    y_col=pdict.get('y_col', 'lat')
    unit=pdict.get('unit', '')
    vmin, vmax = pdict.get('vlim', [0,1])
    cticks=np.linspace(vmin, vmax, pdict.get('nticks', 11))
    cmap=pdict.get('cmap', plt.cm.viridis_r)
    fapply=pdict.get('fapply', None)
    markersize = pdict.get('markersize', 'markersize')
    discrete = pdict.get('discrete', False)

    gdf1 = pandas2geopandas(attrs[[column, x_col, y_col, 'markersize']], x_col=x_col, y_col=y_col)
    if fapply is not None:
        gdf1[column] = gdf1[column].apply(fapply)
    gdf1 = gdf1.sort_values(by=column)
    if isinstance(markersize, str):
        markersize = gdf1[markersize].values

    # main map
    axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
    add_regions(axg, regions, dx=-6, dy=-6)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
    plot_kwargs.update(markersize=markersize, marker='o')
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=discrete,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=f'{column} {unit}', location='right'),
        cbar_pos = dict(pad=0.1, fraction=0.1)
    )
    axg.set_title(column)
    if fapply == np.log10:
        cax.yaxis.set_ticklabels([f'{l:00.0e}' for l in 10**cticks])

    # submaps 
    axs = dict()
    # gdf1 = gdf1.sort_values('markersize', ascending=True)
    for i, (name, bbox) in enumerate(regions.items()):
        ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
        basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
        plot_kwargs.update(markersize=markersize*2, marker='o')
        plot_choropleth(
            fig, ax, gdf1, column=column, 
            cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=discrete,
            plot_kwargs=plot_kwargs, cbar_kwargs=False
        )
        xmin, ymax = bbox[0], bbox[3]
        ax.text(xmin, ymax+1, name, transform=crs_sub)
        axs[name] = ax

    var = column #.split('_')[0]
    if var in list(ds_drivers.data_vars.keys()) and 'ensemble' in ds_drivers[var].dims:
        df_box = ds_drivers[var].to_series().unstack(-1).T
        if fapply is not None:
            df_box = df_box.apply(fapply)
        # boxplot ensemble
        box_kwargs.update(widths=0.5)
        color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
        ax = fig.add_subplot(grid[:-1, -1:])
        df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
        if discrete:
            ax.set_ylim([cticks[1], cticks[-2]])
            ax.yaxis.set_ticks(cticks[1:-2])
        else:
            ax.set_ylim([vmin, vmax])
            ax.yaxis.set_ticks(cticks)
        ax.yaxis.set_ticklabels([])
        ax.grid(axis='x')
        ax.set_xlabel('model')
        cpos = cax.get_position()
        bpos = ax.get_position()
        cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
        # ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
        ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
        cax.yaxis.set_ticks_position('left')
        cax.yaxis.set_label_position('left')
    else:
        cpos = cax.get_position()
        gpos = axg.get_position()
        axg.set_position([gpos.x0+0.1, gpos.y0, gpos.width, gpos.height])
        cax.set_position([cpos.x0+0.05, cpos.y0, cpos.width, cpos.height])

    plt.savefig(join(fdir, f'{column}.png'))
#     plt.close('all')
    print(column)

## frequency

In [ ]:
q = 95
min_dist = 30
Npeaks = 50
window_size=3
fn_peaks_freq = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_top50_all_freq.nc')
ds_freq = xr.open_dataset(fn_peaks_freq)['peak_perc']*100.

dim = 'ensemble'
ds_freq_mean = ds_freq.mean(dim)
# ds_freq_mean
# ds_freq

In [ ]:
vmin, vmax= 0, 100
cticks=np.linspace(vmin, vmax, 11)

for scen in ds_freq['scen'].values:
    label=f'percentage of top {Npeaks} largest peaks [%]'
    gdf = pandas2geopandas(pd.concat([
        attrs[['lat', 'lon', 'markersize']],
        ds_freq_mean.sel(scen=scen).to_series()
    ], axis=1))

    df_box = ds_freq.sel(scen=scen).to_series().unstack(-1).T

    plt.close('all')
    fig = plt.figure(figsize=(17, 10))
    grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

    column = 'peak_perc'
    gdf1 = gdf.sort_values(column, ascending=True)

    # main map
    axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
    add_regions(axg, regions, dx=-6, dy=-6)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
    plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap_turbo, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='right'),
        cbar_pos = dict(pad=0.1, fraction=0.1)
    )
    axg.set_title(scen)

    # submaps 
    axs = dict()
    # gdf1 = gdf1.sort_values('markersize', ascending=True)
    for i, (name, bbox) in enumerate(regions.items()):
        ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
        basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
        plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
        plot_choropleth(
            fig, ax, gdf1, column=column, 
            cmap=cmap_turbo, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs, cbar_kwargs=False
        )
        xmin, ymax = bbox[0], bbox[3]
        ax.text(xmin, ymax+1, name, transform=crs_sub)
        axs[name] = ax

    # boxplot ensemble
    box_kwargs.update(widths=0.5)
    color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
    ax = fig.add_subplot(grid[:-1, -1:])
    df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
    ax.set_ylim([vmin, vmax])
    ax.yaxis.set_ticks(cticks)
    ax.yaxis.set_ticklabels([])
    ax.grid(axis='x')
    ax.set_xlabel('model')
    cpos = cax.get_position()
    bpos = ax.get_position()
    cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
    # ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
    ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
    cax.yaxis.set_ticks_position('left')
    cax.yaxis.set_label_position('left')
    
    
    fn_fig = join(fdir, f'peaks_freq_q{q}d{min_dist}_top{Npeaks}_{scen}.png')
    print(basename(fn_fig))
    plt.savefig(fn_fig)

## rp peak ratio

In [ ]:
q = 95
min_dist = 30
Npeaks = 50
alpha=0.25
method='interp'
# method='gevfit'
fn_peaks_ratio = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp_ratio_{method}.nc')
fn_peaks_rp_ci = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp_ci.nc')
fn_peaks_rp = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp.nc')


ds_ratio = xr.open_dataset(fn_peaks_ratio)
da_wse_rps_ci = xr.open_dataset(fn_peaks_rp_ci)['WSE']
da_wse_rps = xr.open_dataset(fn_peaks_rp)['WSE']

sign_var = [v for v in list(ds_ratio.data_vars.keys()) if v.startswith('sign')]
ds_ratio_sign_var = ds_ratio[sign_var]
ds_ratio = ds_ratio.drop(sign_var)

dim = 'ensemble'
# average and calculate significance based on std error
N = ds_ratio[dim].size
ds_ratio_mean = ds_ratio.mean(dim)
ds_ratio_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_ratio).sum(dim)) == N
ds_ratio_sign = xr.ufuncs.fabs(ds_ratio_mean / ds_ratio.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))

# ds_lst = []
# for var in list(ds_ratio.data_vars.keys()):
#     v1,v2,v3 = var.split('_')
#     da_sign = xr.where(
#         xr.ufuncs.sign(ds_ratio[var])>1,
#         da_wse_rps_ci.sel(scen=v2, alpha=alpha) > da_wse_rps.sel(scen=v3),
#         da_wse_rps_ci.sel(scen=v3, alpha=alpha) > da_wse_rps.sel(scen=v2)
#     ).drop('alpha')
#     da_sign.name = var
#     ds_lst.append(da_sign)
# ds_ratio_sign2 = xr.merge(ds_lst).sum('ensemble') == 1

ds_ratio_stats = xr.concat([
    ds_ratio_mean,
    np.logical_and(ds_ratio_sign, ds_ratio_dir),
], dim='stat').reset_coords(drop=True) #.drop(['diff_surge_seas', 'ratio_surge_seas'])
ds_ratio_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
(ds_ratio_stats.sel(stat='sign', rp=20)==1).sum('index')

In [ ]:
for var in list(ds_ratio_stats.data_vars.keys()):
    if var.startswith('ratio'): continue
    for rp in [2]:
        if var.startswith('ratio'):
            label=f'ratio change return level (rp {rp} year) [-]'
            vmin, vmax, n= -0.5, 3.5, 9
        else:
            label=f'difference return level (rp {rp} year) [m]'
            vmin, vmax, n= -0.2, 0.6, 5
        gdf = pandas2geopandas(pd.concat([
            attrs[['lat', 'lon', 'markersize']],
            ds_ratio_stats[var].sel(rp=rp).to_series().unstack().T
        ], axis=1))

        plt.close('all')
        fig = plt.figure(figsize=(17, 10))
        grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

        column = 'mean'
        idx_sign = gdf['sign']==1

        gdf0 = gdf[idx_sign==False].copy()
        gdf1 = gdf[idx_sign].copy().sort_values(column, ascending=True)
        df_box = ds_ratio[var].sel(rp=rp, index=gdf1.index).to_series().unstack(-1).T

        cticks=np.linspace(vmin, vmax, n)
        cmap_turbo_div = ListedColormap([
            interpolate_cmap(google_turbo_data, x) for x in 
            np.hstack([np.linspace(0.55-(.4*abs(vmin)/vmax)*2, .55, int(-vmin*100)), np.linspace(0.6, 1, int(vmax*100))])
        ])
        # main map
        axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
        add_regions(axg, regions, dx=-6, dy=-6)
        basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
        gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.8), markersize=5, linewidth=0.2, legend=False)

        plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
        cax = plot_choropleth(
            fig, axg, gdf1, column=column, 
            cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs,
            cbar_kwargs=dict(label=label, location='right', extend='both'),
            cbar_pos = dict(pad=0.1, fraction=0.1)
        )

        # submaps 
        axs = dict()
        # gdf1 = gdf1.sort_values('markersize', ascending=True)
        for i, (name, bbox) in enumerate(regions.items()):
            ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
            basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
            gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
            plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
            plot_choropleth(
                fig, ax, gdf1, column=column, 
                cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
                plot_kwargs=plot_kwargs, cbar_kwargs=False
            )
            xmin, ymax = bbox[0], bbox[3]
            ax.text(xmin, ymax+1, name, transform=crs_sub)
            axs[name] = ax

        # boxplot ensemble
        box_kwargs.update(widths=0.5)
        color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
        ax = fig.add_subplot(grid[:-1, -1:])
        df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
#         ax.set_ylim([cticks[1], cticks[-2]])
#         ax.yaxis.set_ticks(cticks[1:-2])
        ax.set_ylim([vmin, vmax])
        ax.yaxis.set_ticks(cticks)
        ax.yaxis.set_ticklabels([])
        ax.grid(axis='x')
        ax.set_xlabel('model')
        cpos = cax.get_position()
        bpos = ax.get_position()
        cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
        # ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
        ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
        cax.yaxis.set_ticks_position('left')
        cax.yaxis.set_label_position('left')

        fn_fig = join(fdir, f'{var}_q{q}d{min_dist}_rp{method}{rp:02d}.png')
        print(basename(fn_fig))
        plt.savefig(fn_fig)
        plt.close('all')
#         break
#     break

## timing

In [ ]:
q = 95
min_dist = 30

fn_peaks_timing = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_timing.nc')
ds_timing = xr.open_dataset(fn_peaks_timing)
ds_timing_diff = ds_timing['doy_diff']


dim = 'ensemble'
# average and calculate significance based on std error
N = ds_timing_diff[dim].size
ds_timing_diff_mean = ds_timing_diff.mean(dim)
ds_timing_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_timing_diff).sum(dim)) == N
ds_timing_diff_sign = xr.ufuncs.fabs(ds_timing_diff_mean / ds_timing_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))
ds_timing_sign = (ds_timing['doy_uniform_p'].max(dim).max('scen')<0.05)

ds_timing_diff_stats = xr.concat([
    ds_timing_diff_mean,
    np.logical_and(np.logical_and(ds_timing_diff_sign, ds_timing_diff_dir), ds_timing_sign),
], dim='stat')
ds_timing_diff_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../3-postprocess/'))

from circ_stats import circ_mean
from temp_stats import doy_to_circ, circ_to_doy

circ = doy_to_circ(ds_timing['doy_mean'], dim=None)
ds_timing_mean = circ_to_doy(circ_mean(circ, dim='ensemble'), None)
ds_timing_sign = ds_timing['doy_uniform_p'].max(dim)<0.05

ds_timing_stats = xr.concat([
    ds_timing_mean,
    ds_timing_sign,
], dim='stat')
ds_timing_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
ds_timing_stats

In [ ]:
vmin, vmax, n= -50, 50, 5
label='change in mean flood day [days]'
cticks=np.linspace(vmin, vmax, n)

gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_diff_stats.to_series().unstack().T
], axis=1))

plt.close('all')
fig = plt.figure(figsize=(17, 10))
grid = plt.GridSpec(2, 6, hspace=0.05, wspace=0.4)

column = 'mean'
idx_sign = gdf['sign']==1

gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
df_box = ds_timing_diff.sel(index=gdf1.index).to_series().unstack(-1).T


# main map
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='.', color='grey', markersize=10, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o', linewidth=0.8)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.1, fraction=0.1)
)

# submaps 
axs = dict()
# gdf1 = gdf1.sort_values('markersize', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='.', color='grey', markersize=10, legend=False)
    plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
    plot_choropleth(
        fig, ax, gdf1, column=column, 
        cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax

# boxplot ensemble
color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
ax = fig.add_subplot(grid[:-1, -1:])
df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
ax.set_ylim([vmin, vmax])
ax.yaxis.set_ticks(cticks)
ax.yaxis.set_ticklabels([])
ax.grid(axis='x')
ax.set_xlabel('model')
cpos = cax.get_position()
bpos = ax.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
# ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')

fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_diff.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')


In [ ]:
amin, amax = -np.pi, np.pi
norm = mpl.colors.Normalize(-np.pi, np.pi)

# legend
dates = pd.date_range(date(2001,1,1), date(2001,12,31), freq='MS')[0::2]
dticks = dates.strftime('%b').tolist()
doys = np.array([d.timetuple().tm_yday for d in dates]).astype(float)
circ = doys/365*2*np.pi # we need to use a [0, 2pi] range here
xval = np.arange(amin, amax, 0.1)
yval = np.ones_like(xval)*2.

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 5))
grid = plt.GridSpec(1, 2, hspace=0.1, wspace=0.1)

# tide
axg = fig.add_subplot(grid[0,0], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='tide').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
# idx_sign = gdf['sign']==1
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy()
# main map
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
# gdf0.plot(ax=axg, marker='.', color='grey', markersize=10, legend=False)
gdf1.plot(ax=axg, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)

# surge
axg2 = fig.add_subplot(grid[0,1], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='surge').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
# idx_sign = gdf['sign']==1
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy()

# main map
basemap(axg2, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
# gdf0.plot(ax=axg2, marker='.', color='grey', markersize=10, legend=False)
gdf1.plot(ax=axg2, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)

# add colorbar
posn = axg.get_position() # fix colorbar position
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1], polar=True) # new ax at random location.
cbar_ax.set_theta_offset(0.5*np.pi)
cbar_ax.set_theta_direction(-1)
cbar_ax.scatter(xval, yval, c=xval, s=45, cmap=cmap_phase, norm=norm, linewidths=0)
cbar_ax.set_yticks([])
cbar_ax.set_xticks(circ)
cbar_ax.set_xticklabels(dticks)
size = posn.height/4.
cbar_ax.set_position([posn.x0-size/8., posn.y0+posn.height/2-size*1.5, size, size])


fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_mean.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
plt.close('all')

## average peak ratio 

top 50 peaks accross all scenarios

In [ ]:
q = 95
min_dist = 30
Npeaks = 50
fn_peaks_ratio = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_top{Npeaks}_all_stat.nc')
ds_ratio = xr.open_dataset(fn_peaks_ratio)

dim = 'ensemble'
# average and calculate significance based on std error
N = ds_ratio[dim].size
ds_ratio_mean = ds_ratio.mean(dim)
ds_ratio_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_ratio).sum(dim)) == N
ds_ratio_sign = xr.ufuncs.fabs(ds_ratio_mean / ds_ratio.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))

ds_ratio_stats = xr.concat([
    ds_ratio_mean,
    np.logical_and(ds_ratio_sign, ds_ratio_dir),
], dim='stat')
ds_ratio_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
ds_ratio_stats

In [ ]:
# vmin, vmax= -1, 1
# cticks=np.linspace(vmin, vmax, 21)

vmin, vmax= -0.2, 0.5
cticks=np.linspace(vmin, vmax, 8)
cmap_turbo_div = ListedColormap([interpolate_cmap(google_turbo_data, x) for x in 
                                 np.hstack([np.linspace(0.34, .5, int(-vmin*100)), np.linspace(0.6, 1, int(vmax*100))])])

for var in ds_ratio.data_vars.keys():
    if var.startswith('ratio'):
        
        gdf = pandas2geopandas(pd.concat([
            attrs[['lat', 'lon', 'markersize']],
            ds_ratio_stats[var].to_series().unstack().T
        ], axis=1))

        df_box = ds_ratio[var].sel(rp=rp).to_series().unstack(-1).T
        plt.close('all')
        fig = plt.figure(figsize=(17, 10))
        grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

        column = 'mean'
        idx_sign = gdf['sign']==1

        gdf0 = gdf[idx_sign==False].copy()
        gdf1 = gdf[idx_sign].copy().sort_values(column, ascending=True)

        # main map
        axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
        add_regions(axg, regions, dx=-6, dy=-6)
        basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
        gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.8), markersize=5, linewidth=0.2, legend=False)

        plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
        cax = plot_choropleth(
            fig, axg, gdf1, column=column, 
            cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs,
            cbar_kwargs=dict(label=f'cfi [-]', location='right', extend='both'),
            cbar_pos = dict(pad=0.1, fraction=0.1)
        )

        # submaps 
        axs = dict()
        # gdf1 = gdf1.sort_values('markersize', ascending=True)
        for i, (name, bbox) in enumerate(regions.items()):
            ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
            basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
            gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
            plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
            plot_choropleth(
                fig, ax, gdf1, column=column, 
                cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
                plot_kwargs=plot_kwargs, cbar_kwargs=False
            )
            xmin, ymax = bbox[0], bbox[3]
            ax.text(xmin, ymax+1, name, transform=crs_sub)
            axs[name] = ax

        # boxplot ensemble
        box_kwargs.update(widths=0.5)
        color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
        ax = fig.add_subplot(grid[:-1, -1:])
        df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
        ax.set_ylim([cticks[1], cticks[-2]])
        ax.yaxis.set_ticks(cticks[1:-2])
        ax.yaxis.set_ticklabels([])
        ax.grid(axis='x')
        ax.set_xlabel('model')
        cpos = cax.get_position()
        bpos = ax.get_position()
        cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
        ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
        cax.yaxis.set_ticks_position('left')
        cax.yaxis.set_label_position('left')

        plt.savefig(join(fdir, f'{var}_q{q}d{min_dist}_top{Npeaks}_all.png'))
        